In [1]:
import random
from tqdm import tqdm_notebook as tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input, Reshape
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling1D, Conv1D
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam, SGD
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [21]:
# First we read in the airline passenger
fileName = "data/international-airline-passengers.csv"
#fileName = "data/wolfer-sunspot-numbers-1770-to-1.csv"

# Load the data
df = pd.read_csv(fileName, engine='python', skipfooter=3)

# Process month to be a time format
print(df.iloc[0][0])
time_column_name = 'Month'
df[time_column_name]=pd.to_datetime(df[time_column_name], format='%Y-%m-%d')
df.set_index([time_column_name], inplace=True)

# Extract the raw data, without the dates
data = df.values
data = data.astype('float32')


# Now we split the data and apply the data scalar
split = 50

train_data =  data[0:split,:]
test_data = data[split:,:]


print("train data", train_data.shape)

def future_data(data,lags=1,future=1):
    X, y = [], []
    for row in range(len(data) - lags - future):
        a = data[row:(row + lags), 0]
        X.append(a)
        y.append(data[row + lags+future-1, 0])
    return np.array(X), np.array(y)

def scale_array(y_vec):
    
    ymax = np.max(y_vec)
    ymin = np.min(y_vec)
    
    print("ymax",ymax)
    print("ymin",ymin)
    
    y_vec_scaled = np.zeros((len(y_vec),1))
    
    for k in range(0,len(y_vec)):
        y_vec_scaled[k][0] = (y_vec[k][0]-ymin)/(ymax-ymin)
    
    return y_vec_scaled,ymin,ymax

def invert_scaling(y_vec_scaled,ymin,ymax):
    
    y_vec = np.zeros(y_vec_scaled.shape)
        
        
    for k in range(0,len(y_vec_scaled)):
        y_vec[k] = ymin+(ymax-ymin)*y_vec_scaled[k]
    
    return y_vec

# Process the train and test data
train_data,train_min,train_max = scale_array(train_data)
test_data,test_min,test_max = scale_array(test_data)

lags=1
future=1

# Process the training data
X_train, y_train = future_data(train_data, lags,future)
X_test, y_test = future_data(test_data, lags,future)

print(X_train.shape)

1949-01
train data (50, 1)
ymax 242.0
ymin 104.0
ymax 622.0
ymin 180.0
(48, 1)


In [25]:
# Here we define the generative model

def get_generative(G_in, dense_dim=200, lr=1e-3):
    '''
    This model generates the data
    '''
    x = Dense(dense_dim)(G_in)
    x = Activation('tanh')(x)
    G_out = Dense(X_train.shape[0], activation='tanh')(x)
    G = Model(G_in, G_out)
    opt = SGD(lr=lr)
    G.compile(loss='binary_crossentropy', optimizer=opt)
    return G, G_out

G_in = Input(X_train.shape)
G, G_out = get_generative(G_in)
G.summary()


def get_discriminative(D_in, lr=1e-3, drate=.25, conv_sz=5, leak=.2):
    x = Reshape((-1, 1))(D_in)
    x = Conv1D(X_train.shape[0], conv_sz, activation='relu')(x)
    x = Dropout(drate)(x)
    x = Flatten()(x)
    x = Dense(X_train.shape[0])(x)
    D_out = Dense(2, activation='sigmoid')(x)
    D = Model(D_in, D_out)
    dopt = Adam(lr=lr)
    D.compile(loss='binary_crossentropy', optimizer=dopt)
    return D, D_out

D_in = Input(X_train.shape)
D, D_out = get_discriminative(D_in)
D.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        (None, 48, 1)             0         
_________________________________________________________________
dense_29 (Dense)             (None, 48, 200)           400       
_________________________________________________________________
activation_10 (Activation)   (None, 48, 200)           0         
_________________________________________________________________
dense_30 (Dense)             (None, 48, 48)            9648      
Total params: 10,048
Trainable params: 10,048
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        (None, 48, 1)             0         
_________________________________________________________________
reshap

In [27]:
def set_trainability(model, trainable=False):
    model.trainable = trainable
    for layer in model.layers:
        layer.trainable = trainable
        
def make_gan(GAN_in, G, D):
    set_trainability(D, False)
    x = G(GAN_in)
    GAN_out = D(x)
    GAN = Model(GAN_in, GAN_out)
    GAN.compile(loss='binary_crossentropy', optimizer=G.optimizer)
    return GAN, GAN_out

GAN_in = Input(X_train.shape)
GAN, GAN_out = make_gan(GAN_in, G, D)
GAN.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        (None, 48, 1)             0         
_________________________________________________________________
model_14 (Model)             (None, 48, 48)            10048     
_________________________________________________________________
model_15 (Model)             (None, 2)                 101810    
Total params: 111,858
Trainable params: 10,048
Non-trainable params: 101,810
_________________________________________________________________


In [28]:
def sample_data_and_gen(G, noise_dim=10, n_samples=10000):
    XT = X_train#sample_data(n_samples=n_samples)
    XN_noise = np.random.uniform(0, 1, size=[n_samples, noise_dim])
    XN = G.predict(XN_noise)
    X = np.concatenate((XT, XN))
    y = np.zeros((2*n_samples, 2))
    y[:n_samples, 1] = 1
    y[n_samples:, 0] = 1
    return X, y

def pretrain(G, D, noise_dim=10, n_samples=10000, batch_size=32):
    X, y = sample_data_and_gen(G, n_samples=n_samples, noise_dim=noise_dim)
    set_trainability(D, True)
    D.fit(X, y, epochs=1, batch_size=batch_size)

pretrain(G, D)

ValueError: Error when checking input: expected input_17 to have 3 dimensions, but got array with shape (10000, 10)